In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math


In [4]:
import random
import numpy as np
from numpy import ndarray

In [5]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [6]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [7]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (MB, SQ), (SB, MQ), (MB, MQ), (SB, SQ), (MB, SQ), (LB, LQ), (SB, LQ),
     (SB, SQ), (MB, MQ), (MB, SQ), (MB, MQ), (MB, SQ), (SB, SQ), (MB, MQ),
      (MB, LQ), (SB, MQ), (MB, SQ), (MB, MQ), (SB, SQ), (SB, MQ)
 ]

In [20]:
# Compute total number of required points across all test combinations
required_points = sum(data_size for data_size, _ in test_combinations)

# Sample from the full dataset without replacement
if required_points > total_points:
    raise ValueError(f"Need {required_points} points but only {total_points} available.")

sampled_indices = np.random.choice(total_points, size=required_points, replace=False)
combined_dataset = dataset[sampled_indices]


In [21]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


In [ ]:
print(len(queries_list))
print(len(combined_dataset))

20
2180000


**-------------Cover Tree---------------**

**Izbicki's implementation**

In [ ]:
import numpy as np
from typing import Optional, List, Tuple
from dataclasses import dataclass

@dataclass
class Node:
    point: np.ndarray
    level: int
    children: List['Node']
    _maxdist_cache: Optional[float] = None  # Cache for maxdist

    @property
    def covdist(self) -> float:
        """Coverage distance of the node."""
        return 2.0 ** self.level

    @property
    def maxdist(self) -> float:
        """Maximum distance within the subtree rooted at this node."""
        if self._maxdist_cache is None:
            if not self.children:
                self._maxdist_cache = 0
            else:
                self._maxdist_cache = max(
                    self.covdist,
                    max(child.maxdist + np.linalg.norm(self.point - child.point)
                        for child in self.children)
                )
        return self._maxdist_cache

    def update_maxdist(self):
        """Reset and recompute maxdist cache."""
        self._maxdist_cache = None
        _ = self.maxdist  # Force recomputation

class CoverTree:
    def __init__(self):
        self.root: Optional[Node] = None

    def insert(self, x: np.ndarray) -> None:
        """Insert a point into the cover tree."""
        if self.root is None:
            self.root = Node(x, 0, [])
            return

        d = np.linalg.norm(self.root.point - x)
        if d > self.root.covdist:
            # Precompute levels to expand
            levels_to_expand = int(np.ceil(np.log2(d / self.root.covdist)))
            for _ in range(levels_to_expand):
                self.root = Node(self.root.point, self.root.level + 1, [self.root])
            self.root = Node(x, self.root.level + 1, [self.root])
            return

        self._insert_recursive(self.root, x)

    def _insert_recursive(self, p: Node, x: np.ndarray) -> bool:
        """Recursive insertion into the cover tree."""
        for child in p.children:
            if np.linalg.norm(child.point - x) <= child.covdist:
                if self._insert_recursive(child, x):
                    p.update_maxdist()
                    return True

        # If x fits within the current node's coverage
        if np.linalg.norm(p.point - x) <= p.covdist:
            p.children.append(Node(x, p.level - 1, []))
            p.update_maxdist()
            return True

        return False

    def find_nearest(self, x: np.ndarray) -> Tuple[np.ndarray, float]:
        """Find the nearest neighbor to the query point."""
        if self.root is None:
            raise ValueError("Tree is empty")

        return self._find_nearest_recursive(self.root, x, self.root)

    def _find_nearest_recursive(self, p: Node, x: np.ndarray, best: Node) -> Tuple[np.ndarray, float]:
        """Recursive nearest neighbor search with pruning."""
        d_p = np.linalg.norm(p.point - x)
        d_best = np.linalg.norm(best.point - x)

        # Update the best point if the current node is closer
        if d_p < d_best:
            best = p
            d_best = d_p

        # Sort children by distance to the query point
        for child in sorted(p.children, key=lambda c: np.linalg.norm(c.point - x)):
            d_x_to_child = np.linalg.norm(child.point - x)

            # Prune subtrees based on the pruning condition
            if d_best > d_x_to_child - child.maxdist:
                child_point, child_dist = self._find_nearest_recursive(child, x, best)
                if child_dist < d_best:
                    best = Node(child_point, p.level, [])
                    d_best = child_dist

        return best.point, d_best


-----------

**----------CV Run 1----------------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 32.74it/s]


Build Time: 85.27 sec, Search Time: 0.61 sec

1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:03<00:00, 31.67it/s]


Build Time: 10.31 sec, Search Time: 3.14 sec

2: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 185.25 sec, Search Time: 12.77 sec

18: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:03<00:00,  5.63it/s]


Build Time: 18.45 sec, Search Time: 3.54 sec

19: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:10<00:00,  9.47it/s]

Build Time: 18.75 sec, Search Time: 10.51 sec

=== Summary of CV Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 85.27 sec, Search: 0.61 sec
Data:   10000 points, Queries:  100 => Build: 10.31 sec, Search: 3.14 sec
Data:  100000 points, Queries:  100 => Build: 111.65 sec, Search: 4.76 sec
Data:   10000 points, Queries:   20 => Build: 11.88 sec, Search: 0.96 sec
Data:  100000 points, Queries:   20 => Build: 124.61 sec, Search: 1.05 sec
Data: 1000000 points, Queries:  500 => Build: 1542.57 sec, Search: 49.17 sec
Data:   10000 points, Queries:  500 => Build: 16.85 sec, Search: 48.77 sec
Data:   10000 points, Queries:   20 => Build: 17.09 sec, Search: 1.71 sec
Data:  100000 points, Queries:  100 => Build: 171.57 sec, Search: 10.22 sec
Data:  100000 points, Queries:   20 => Build: 172.93 sec, Search: 2.34 sec
Data:  100000 points, Queries:  100 => Build: 175.02 sec, Search: 10.78 sec
Data:  100000 points, Queries:   20 => Build: 175.34 sec, Search: 2.23 sec
Data:   10000 points

**-------------CV Run 2-----------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 34.03it/s]


Build Time: 80.99 sec, Search Time: 0.58 sec

1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:02<00:00, 35.42it/s]


Build Time: 9.93 sec, Search Time: 2.80 sec

2: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 182.73 sec, Search Time: 12.06 sec

18: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:01<00:00, 10.33it/s]


Build Time: 18.47 sec, Search Time: 1.93 sec

19: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:13<00:00,  7.21it/s]

Build Time: 19.12 sec, Search Time: 13.79 sec

=== Summary of CV Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 80.99 sec, Search: 0.58 sec
Data:   10000 points, Queries:  100 => Build: 9.93 sec, Search: 2.80 sec
Data:  100000 points, Queries:  100 => Build: 108.12 sec, Search: 4.57 sec
Data:   10000 points, Queries:   20 => Build: 11.39 sec, Search: 1.03 sec
Data:  100000 points, Queries:   20 => Build: 122.37 sec, Search: 0.98 sec
Data: 1000000 points, Queries:  500 => Build: 1530.47 sec, Search: 48.57 sec
Data:   10000 points, Queries:  500 => Build: 17.27 sec, Search: 47.45 sec
Data:   10000 points, Queries:   20 => Build: 16.93 sec, Search: 2.00 sec
Data:  100000 points, Queries:  100 => Build: 169.11 sec, Search: 10.14 sec
Data:  100000 points, Queries:   20 => Build: 174.82 sec, Search: 1.91 sec
Data:  100000 points, Queries:  100 => Build: 172.26 sec, Search: 10.61 sec
Data:  100000 points, Queries:   20 => Build: 171.50 sec, Search: 1.64 sec
Data:   10000 points,

**-----------CV Analysis------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 3661.44',
    'Total Time: 3621.70'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 3641.5700 sec, Std Dev: 28.1004 sec, CV: 0.77%


--------------

**--------------------------VP_MWV-----------------------------**

In [ ]:
import numpy as np

class VPNode:
    def __init__(self, vp=None):
        # Vantage point
        self.vp = vp

        # Distance bounds
        self.low_near = float('inf')
        self.low_far = 0
        self.high_near = float('inf')
        self.high_far = 0

        # Child nodes
        self.lowChild = None
        self.highChild = None

        # Bucket for leaf nodes
        self.bucket = []

    def is_leaf(self):
        return self.lowChild is None and self.highChild is None

class VPTree:
    def __init__(self, bucket_size=50, distance_fn=None):
        """
        Initialize a VP-Tree

        Parameters:
        - bucket_size: Square root of maximum bucket size (b where b² is max bucket size)
        - distance_fn: Distance function to use (defaults to Euclidean)
        """
        self.bucket_size = bucket_size
        self.root = None

        # Default to Euclidean distance if none provided
        self.distance_fn = distance_fn if distance_fn is not None else self._euclidean_distance

    def _euclidean_distance(self, x, y):
        """Calculate Euclidean distance between two points"""
        x = np.asarray(x)
        y = np.asarray(y)
        diff = x - y
        return np.sqrt(np.dot(diff, diff))

    def insert(self, x):
        """Insert a new datapoint x into the VP-tree"""
        # If tree is empty, create root with this point as VP
        if self.root is None:
            self.root = VPNode(vp=x)
            return

        p = self.root

        # Traverse until we reach a leaf node
        while not p.is_leaf():
            # Calculate distance from x to current vantage point
            dist = self.distance_fn(x, p.vp)

            # Determine which child to go to
            if dist < (p.low_far + p.high_near) / 2:
                # Update distance bounds
                p.low_far = max(dist, p.low_far)
                p.low_near = min(dist, p.low_near)

                # Go to low (near) child
                p = p.lowChild
            else:
                # Update distance bounds
                p.high_far = max(dist, p.high_far)
                p.high_near = min(dist, p.high_near)

                # Go to high (far) child
                p = p.highChild

        # Add x to bucket leaf node p
        p.bucket.append(x)

        # If bucket exceeds b², split it
        if len(p.bucket) > self.bucket_size**2:
            self._split_node(p)

    def _select_vantage_point(self, points):
        """Select a vantage point from the bucket
        Can use various methods; simple random selection is shown here.
        """
        # For simplicity, using the first point as VP
        # Could implement more sophisticated selection methods
        if points:
            return points[0]
        return None

    def _split_node(self, p):
        """Split a node using the CART-inspired weighted variance method"""
        points = p.bucket.copy()

        # Select vantage point from bucket
        vp = self._select_vantage_point(points)
        points.remove(vp)  # Remove VP from points list

        # Calculate distances to vantage point
        distances = [(self.distance_fn(vp, point), point) for point in points]

        # Sort by distance
        distances.sort(key=lambda x: x[0])
        sorted_points = [point for _, point in distances]
        sorted_dists = [dist for dist, _ in distances]

        # Find optimal split using weighted variance minimization
        n = len(sorted_points)
        best_split = 0
        min_weighted_variance = float('inf')

        for s in range(1, n):
            # Calculate variances for 1:s and s:n
            if s > 0:
                var_left = np.var(sorted_dists[:s]) if s > 1 else 0
            else:
                var_left = 0

            if n-s > 0:
                var_right = np.var(sorted_dists[s:]) if n-s > 1 else 0
            else:
                var_right = 0

            # Calculate weighted variance
            weighted_var = s * var_left + (n-s) * var_right

            if weighted_var < min_weighted_variance:
                min_weighted_variance = weighted_var
                best_split = s

        # Create new children nodes
        p.lowChild = VPNode(vp=vp)
        p.highChild = VPNode(vp=self._select_vantage_point(sorted_points[best_split:]))

        # Distribute points to children based on best split
        p.lowChild.bucket = sorted_points[:best_split]
        p.highChild.bucket = sorted_points[best_split:]

        # Initialize distance bounds
        if p.lowChild.bucket:
            low_dists = sorted_dists[:best_split]
            p.low_near = min(low_dists)
            p.low_far = max(low_dists)

        if p.highChild.bucket:
            # If we have a vantage point for the high child, calculate distances to it
            high_dists = [self.distance_fn(p.highChild.vp, point) for point in p.highChild.bucket]
            p.high_near = min(high_dists) if high_dists else float('inf')
            p.high_far = max(high_dists) if high_dists else 0

        # Clear the bucket as it's no longer a leaf
        p.bucket = []

    def nearest_neighbor(self, query):
        """Optimized nearest neighbor search with batch processing for bucket points"""
        best_dist = float('inf')
        best_point = None

        # Use a stack instead of recursion to reduce overhead
        # Each entry is (node, min_possible_dist)
        stack = [(self.root, 0)]

        while stack:
            node, min_dist = stack.pop()

            # Skip this node if it cannot contain a better solution
            if min_dist >= best_dist:
                continue

            # Check vantage point distance
            if node.vp is not None:
                # Calculate distance to vantage point
                diff = np.subtract(query, node.vp)
                dist_to_vp = np.sqrt(np.dot(diff, diff))

                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.vp

            # If leaf node, check all points in bucket using batch processing
            if node.is_leaf():
                if node.bucket:
                    # Convert bucket to numpy array once
                    bucket_array = np.array(node.bucket)

                    # Vectorized distance calculation
                    query_array = np.array(query)
                    diffs = bucket_array - query_array
                    squared_dists = np.sum(diffs**2, axis=1)

                    # Find minimum distance point without unnecessary sqrt operations
                    min_idx = np.argmin(squared_dists)
                    min_squared_dist = squared_dists[min_idx]

                    # Only compute sqrt for potentially better distance
                    if min_squared_dist < best_dist**2:
                        min_dist = np.sqrt(min_squared_dist)
                        if min_dist < best_dist:
                            best_dist = min_dist
                            best_point = node.bucket[min_idx]
                continue

            # Calculate threshold
            threshold = (node.low_far + node.high_near) / 2

            # Calculate bounds for both children
            if dist_to_vp < threshold:
                # Query is in near region
                # Add far child to stack first (so near child gets processed first)
                if node.highChild:
                    far_bound = max(0, dist_to_vp - node.low_far)
                    if far_bound < best_dist:
                        stack.append((node.highChild, far_bound))

                # Add near child with zero bound (must contain query)
                if node.lowChild:
                    stack.append((node.lowChild, 0))
            else:
                # Query is in far region
                # Add near child to stack first (so far child gets processed first)
                if node.lowChild:
                    near_bound = max(0, node.high_near - dist_to_vp)
                    if near_bound < best_dist:
                        stack.append((node.lowChild, near_bound))

                # Add far child with zero bound (must contain query)
                if node.highChild:
                    stack.append((node.highChild, 0))

        return best_dist, best_point

----------

**VP_MWV (Run 1)**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 126.56it/s]


Build Time: 24.04 sec, Search Time: 0.16 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 1084.99it/s]


Build Time: 3.25 sec, Search Time: 0.09 sec

3: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 35.58 sec, Search Time: 0.14 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 597.87it/s]


Build Time: 3.41 sec, Search Time: 0.03 sec

20: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 613.29it/s]

Build Time: 3.27 sec, Search Time: 0.16 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 24.04 sec, Search: 0.16 sec
Data:   10000 points, Queries:  100 => Build: 3.25 sec, Search: 0.09 sec
Data:  100000 points, Queries:  100 => Build: 26.60 sec, Search: 0.17 sec
Data:   10000 points, Queries:   20 => Build: 2.41 sec, Search: 0.02 sec
Data:  100000 points, Queries:   20 => Build: 27.41 sec, Search: 0.03 sec
Data: 1000000 points, Queries:  500 => Build: 305.20 sec, Search: 2.65 sec
Data:   10000 points, Queries:  500 => Build: 1.40 sec, Search: 0.77 sec
Data:   10000 points, Queries:   20 => Build: 1.77 sec, Search: 0.04 sec
Data:  100000 points, Queries:  100 => Build: 32.38 sec, Search: 0.15 sec
Data:  100000 points, Queries:   20 => Build: 33.09 sec, Search: 0.04 sec
Data:  100000 points, Queries:  100 => Build: 32.34 sec, Search: 0.12 sec
Data:  100000 points, Queries:   20 => Build: 34.48 sec, Search: 0.04 sec
Data:   10000 points, Queries:   20 => B

**-----VP_MWV Run 2-------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 148.66it/s]


Build Time: 23.88 sec, Search Time: 0.13 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 727.76it/s]


Build Time: 3.73 sec, Search Time: 0.14 sec

3: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 35.41 sec, Search Time: 0.14 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 639.22it/s]


Build Time: 2.75 sec, Search Time: 0.03 sec

20: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 595.26it/s]

Build Time: 4.66 sec, Search Time: 0.17 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 23.88 sec, Search: 0.13 sec
Data:   10000 points, Queries:  100 => Build: 3.73 sec, Search: 0.14 sec
Data:  100000 points, Queries:  100 => Build: 27.53 sec, Search: 0.24 sec
Data:   10000 points, Queries:   20 => Build: 2.81 sec, Search: 0.03 sec
Data:  100000 points, Queries:   20 => Build: 28.29 sec, Search: 0.04 sec
Data: 1000000 points, Queries:  500 => Build: 313.26 sec, Search: 2.06 sec
Data:   10000 points, Queries:  500 => Build: 1.76 sec, Search: 0.66 sec
Data:   10000 points, Queries:   20 => Build: 2.93 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 35.09 sec, Search: 0.20 sec
Data:  100000 points, Queries:   20 => Build: 27.51 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 34.31 sec, Search: 0.14 sec
Data:  100000 points, Queries:   20 => Build: 34.10 sec, Search: 0.04 sec
Data:   10000 points, Queries:   20 => B

**-------VP_MWV Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 126.80it/s]


Build Time: 23.33 sec, Search Time: 0.16 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 807.26it/s]


Build Time: 2.60 sec, Search Time: 0.12 sec

3: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 33.89 sec, Search Time: 0.14 sec

19: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 485.49it/s]


Build Time: 5.17 sec, Search Time: 0.04 sec

20: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 718.74it/s]

Build Time: 3.82 sec, Search Time: 0.14 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 23.33 sec, Search: 0.16 sec
Data:   10000 points, Queries:  100 => Build: 2.60 sec, Search: 0.12 sec
Data:  100000 points, Queries:  100 => Build: 28.83 sec, Search: 0.21 sec
Data:   10000 points, Queries:   20 => Build: 3.12 sec, Search: 0.02 sec
Data:  100000 points, Queries:   20 => Build: 25.92 sec, Search: 0.04 sec
Data: 1000000 points, Queries:  500 => Build: 301.87 sec, Search: 2.05 sec
Data:   10000 points, Queries:  500 => Build: 2.72 sec, Search: 0.81 sec
Data:   10000 points, Queries:   20 => Build: 2.45 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 35.17 sec, Search: 0.16 sec
Data:  100000 points, Queries:   20 => Build: 32.58 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 30.98 sec, Search: 0.17 sec
Data:  100000 points, Queries:   20 => Build: 28.15 sec, Search: 0.04 sec
Data:   10000 points, Queries:   20 => B

**---------VP MWV Analysis---------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 671.77',
    'Total Time: 693.55',
    'Total Time: 669.84'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 678.3867 sec, Std Dev: 13.1672 sec, CV: 1.94%


-----------------

**------------------Ball Tree--------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=40):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
    #*-----------------------------------------------------------------------------

    def insert(self, point):
        """
        Insert a new point into the Ball Tree.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        # Path to keep track of nodes to update
        path = []

        # Traverse the tree to find the appropriate leaf
        current = self.root
        path.append(current)

        while current.points is None:  # Continue until we reach a leaf node
            # Calculate distances to pivots
            dist_to_p1 = np.sqrt(np.sum((point - current.pivot1) ** 2))
            dist_to_p2 = np.sqrt(np.sum((point - current.pivot2) ** 2))

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid)
        # New centroid = (n * old_centroid + new_point) / (n + 1)
        current.center = (n * current.center + point) / (n + 1)

        # Update the radius if necessary
        dist_to_center = np.sqrt(np.sum((point - current.center) ** 2))
        current.radius = max(current.radius, dist_to_center)

        # Update centers and radii of ancestor nodes
        path.pop()  # Remove the leaf node as we've already updated it
        self._update_ancestors(path, point)

    #*-----------------------------------------------------------------------------
    def _update_ancestors(self, path, point):
        """
        Update the center and radius of all nodes in the path after insertion.

        Parameters:
            path (list): List of nodes from root to leaf (excluding the leaf)
            point (np.ndarray): The point that was inserted
        """
        # Process nodes from bottom to top (leaf's parent first)
        for node in reversed(path):
            # For center, we would ideally recompute based on all points
            # but as an approximation, we'll slightly shift it toward the new point
            # This way we avoid having to recursively gather all points

            # We'll apply a small weight to the new point based on estimated subtree size
            # Assume a balanced tree with approximately 2^depth nodes at the bottom
            depth = len(path) - path.index(node)
            weight = 1.0 / (2**depth)

            # Update center with minimal shift toward new point
            node.center = (1 - weight) * node.center + weight * point

            # Check if radius needs update by calculating distance from new point to center
            dist_to_center = np.sqrt(np.sum((point - node.center) ** 2))

            # Only update radius if the new point is outside the current ball
            if dist_to_center > node.radius:
                node.radius = dist_to_center

    #*-----------------------------------------------------------------------------
    def insert_opt(self, point):
        """
        Insert a new point into the Ball Tree with optimized performance.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        current = self.root
        nodes_to_update = []  # Store nodes that need radius updates

        # Traverse the tree to find the appropriate leaf
        while current.points is None:  # Continue until we reach a leaf node
            # Check if point is within current node's radius
            dist_to_center = np.linalg.norm(point - current.center)
            if dist_to_center > current.radius:
                nodes_to_update.append((current, dist_to_center))

            # Calculate distances to pivots (only once per level)
            dist_to_p1 = np.linalg.norm(point - current.pivot1)
            dist_to_p2 = np.linalg.norm(point - current.pivot2)

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid) - avoid creating temporary arrays
        current.center *= n
        current.center += point
        current.center /= (n + 1)

        # Update the radius if necessary
        dist_to_center = np.linalg.norm(point - current.center)
        if dist_to_center > current.radius:
            current.radius = dist_to_center

        # Update only radii of nodes that need updates
        for node, dist in nodes_to_update:
            node.radius = max(node.radius, dist)
#*-----------------------------------------------------------------------------

-----------

**--------Ball Tree (Run 1)----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 650

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 443.31it/s]


Build Time: 0.24 sec, Search Time: 0.04 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 396.17it/s]


Build Time: 0.25 sec, Search Time: 0.25 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 7.50 sec, Search Time: 0.75 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 130.09it/s]


Build Time: 7.45 sec, Search Time: 0.15 sec

20:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 124.31it/s]

Build Time: 7.50 sec, Search Time: 0.80 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 0.24 sec, Search: 0.04 sec
Data:   10000 points, Queries:  100 => Build: 0.25 sec, Search: 0.25 sec
Data:  100000 points, Queries:  100 => Build: 0.52 sec, Search: 0.33 sec
Data:   10000 points, Queries:   20 => Build: 0.55 sec, Search: 0.07 sec
Data:  100000 points, Queries:   20 => Build: 0.84 sec, Search: 0.07 sec
Data: 1000000 points, Queries:  500 => Build: 4.47 sec, Search: 3.58 sec
Data:   10000 points, Queries:  500 => Build: 4.31 sec, Search: 3.67 sec
Data:   10000 points, Queries:   20 => Build: 4.33 sec, Search: 0.15 sec
Data:  100000 points, Queries:  100 => Build: 4.72 sec, Search: 0.81 sec
Data:  100000 points, Queries:   20 => Build: 5.10 sec, Search: 0.12 sec
Data:  100000 points, Queries:  100 => Build: 5.40 sec, Search: 0.74 sec
Data:  100000 points, Queries:   20 => Build: 5.83 sec, Search: 0.18 sec
Data:   10000 points, Queries:   20 => Build: 5.8

**-------Ball Run 2----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 650

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 426.25it/s]


Build Time: 0.23 sec, Search Time: 0.05 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 436.24it/s]


Build Time: 0.25 sec, Search Time: 0.23 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 7.56 sec, Search Time: 0.69 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 131.30it/s]


Build Time: 7.58 sec, Search Time: 0.15 sec

20:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 108.53it/s]

Build Time: 7.61 sec, Search Time: 0.91 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 0.23 sec, Search: 0.05 sec
Data:   10000 points, Queries:  100 => Build: 0.25 sec, Search: 0.23 sec
Data:  100000 points, Queries:  100 => Build: 0.51 sec, Search: 0.36 sec
Data:   10000 points, Queries:   20 => Build: 0.54 sec, Search: 0.07 sec
Data:  100000 points, Queries:   20 => Build: 0.81 sec, Search: 0.08 sec
Data: 1000000 points, Queries:  500 => Build: 4.31 sec, Search: 3.61 sec
Data:   10000 points, Queries:  500 => Build: 4.31 sec, Search: 3.56 sec
Data:   10000 points, Queries:   20 => Build: 4.33 sec, Search: 0.14 sec
Data:  100000 points, Queries:  100 => Build: 4.71 sec, Search: 0.77 sec
Data:  100000 points, Queries:   20 => Build: 5.07 sec, Search: 0.13 sec
Data:  100000 points, Queries:  100 => Build: 5.52 sec, Search: 0.78 sec
Data:  100000 points, Queries:   20 => Build: 5.90 sec, Search: 0.14 sec
Data:   10000 points, Queries:   20 => Build: 5.9

**---------Ball Tree Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 650

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 429.22it/s]


Build Time: 0.24 sec, Search Time: 0.05 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 410.63it/s]


Build Time: 0.25 sec, Search Time: 0.24 sec

3:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


...

Build Time: 7.55 sec, Search Time: 0.79 sec

19:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 117.74it/s]


Build Time: 7.52 sec, Search Time: 0.17 sec

20:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 126.40it/s]

Build Time: 7.63 sec, Search Time: 0.78 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 0.24 sec, Search: 0.05 sec
Data:   10000 points, Queries:  100 => Build: 0.25 sec, Search: 0.24 sec
Data:  100000 points, Queries:  100 => Build: 0.54 sec, Search: 0.33 sec
Data:   10000 points, Queries:   20 => Build: 0.55 sec, Search: 0.07 sec
Data:  100000 points, Queries:   20 => Build: 0.83 sec, Search: 0.08 sec
Data: 1000000 points, Queries:  500 => Build: 4.31 sec, Search: 3.47 sec
Data:   10000 points, Queries:  500 => Build: 4.35 sec, Search: 3.50 sec
Data:   10000 points, Queries:   20 => Build: 4.39 sec, Search: 0.15 sec
Data:  100000 points, Queries:  100 => Build: 4.75 sec, Search: 0.75 sec
Data:  100000 points, Queries:   20 => Build: 5.08 sec, Search: 0.15 sec
Data:  100000 points, Queries:  100 => Build: 5.53 sec, Search: 0.74 sec
Data:  100000 points, Queries:   20 => Build: 5.86 sec, Search: 0.16 sec
Data:   10000 points, Queries:   20 => Build: 5.8

**---------Ball Tree Analysis---------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 109.16',
    'Total Time: 109.75'
    'Total Time: 109.23'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 109.4550 sec, Std Dev: 0.4172 sec, CV: 0.38%


----------

**----------QuickSelect Algorithm-----------**

In [10]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

-----------------------------------

-------------------------------------------------------

**--------KD Hybrid---------**

In [11]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

-------------------------------

**----------KD Hybrid Sqrt Run 1-----------**

In [12]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.6590 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 721.99it/s]


Total Search Time: 0.059413 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 100612.51it/s]


Insert Time: 0.1026 sec


...

Total Search Time: 0.996094 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 66037.89it/s]


Insert Time: 0.1547 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 91.04it/s]

Total Search Time: 1.886130 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.6590 sec, Total Search Time: 0.059413 sec, Total Time: 0.7184 sec
Points: 110,000, Insert Time: 0.1026 sec, Total Search Time: 0.177078 sec, Total Time: 0.2797 sec
Points: 210,000, Build Time: 1.4465 sec, Total Search Time: 0.271928 sec, Total Time: 1.7184 sec
Points: 220,000, Insert Time: 0.1160 sec, Total Search Time: 0.101638 sec, Total Time: 0.2176 sec
Points: 320,000, Insert Time: 1.0022 sec, Total Search Time: 0.199178 sec, Total Time: 1.2013 sec
Points: 1,320,000, Build Time: 11.7159 sec, Total Search Time: 5.423155 sec, Total Time: 17.1391 sec
Points: 1,330,000, Insert Time: 0.1229 sec, Total Search Time: 5.547465 sec, Total Time: 5.6704 sec
Points: 1,340,000, Insert Time: 0.1067 sec, Total Search Time: 0.505286 sec, Total Time: 0.6120 sec
Points: 1,440,000, Insert Time: 1.0531 sec, Total Search Time: 1.373222 sec, Total Time: 2.4264 sec
Points: 1,540,000, Insert Time: 1.2390

**---------KD Hybrid Sqrt Run 2-------------**

In [17]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.9983 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 727.65it/s]


Total Search Time: 0.056874 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 112724.96it/s]


Insert Time: 0.0910 sec


...

Total Search Time: 1.059322 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 67715.16it/s]


Insert Time: 0.1498 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 87.82it/s]

Total Search Time: 2.033780 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.9983 sec, Total Search Time: 0.056874 sec, Total Time: 1.0552 sec
Points: 110,000, Insert Time: 0.0910 sec, Total Search Time: 0.162360 sec, Total Time: 0.2533 sec
Points: 210,000, Build Time: 1.5067 sec, Total Search Time: 0.273297 sec, Total Time: 1.7800 sec
Points: 220,000, Insert Time: 0.1081 sec, Total Search Time: 0.107887 sec, Total Time: 0.2160 sec
Points: 320,000, Insert Time: 0.8630 sec, Total Search Time: 0.170992 sec, Total Time: 1.0340 sec
Points: 1,320,000, Build Time: 11.7952 sec, Total Search Time: 5.440294 sec, Total Time: 17.2355 sec
Points: 1,330,000, Insert Time: 0.1277 sec, Total Search Time: 5.500513 sec, Total Time: 5.6282 sec
Points: 1,340,000, Insert Time: 0.1002 sec, Total Search Time: 0.530958 sec, Total Time: 0.6311 sec
Points: 1,440,000, Insert Time: 1.0505 sec, Total Search Time: 1.373870 sec, Total Time: 2.4244 sec
Points: 1,540,000, Insert Time: 1.0161

**----------KD Hybrid Sqrt Run 3--------------**

In [22]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 1.1018 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 859.08it/s]


Total Search Time: 0.053583 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 127161.00it/s]


Insert Time: 0.0808 sec


...

Total Search Time: 0.955339 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 72022.16it/s]


Insert Time: 0.1411 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 95.55it/s]

Total Search Time: 1.818483 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 1.1018 sec, Total Search Time: 0.053583 sec, Total Time: 1.1554 sec
Points: 110,000, Insert Time: 0.0808 sec, Total Search Time: 0.169180 sec, Total Time: 0.2500 sec
Points: 210,000, Build Time: 1.5529 sec, Total Search Time: 0.284775 sec, Total Time: 1.8377 sec
Points: 220,000, Insert Time: 0.0792 sec, Total Search Time: 0.119932 sec, Total Time: 0.1991 sec
Points: 320,000, Insert Time: 0.9575 sec, Total Search Time: 0.180796 sec, Total Time: 1.1383 sec
Points: 1,320,000, Build Time: 11.9416 sec, Total Search Time: 5.404606 sec, Total Time: 17.3463 sec
Points: 1,330,000, Insert Time: 0.1474 sec, Total Search Time: 5.501597 sec, Total Time: 5.6490 sec
Points: 1,340,000, Insert Time: 0.1223 sec, Total Search Time: 0.544991 sec, Total Time: 0.6673 sec
Points: 1,440,000, Insert Time: 1.1406 sec, Total Search Time: 1.447305 sec, Total Time: 2.5879 sec
Points: 1,540,000, Insert Time: 1.1194

**-----------KD Hybrid Sqrt Analysis-----------**

In [27]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time:: 57.1429',
    'Total Time:: 56.9514',
    'Total Time:: 57.3487'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.1477 sec, Std Dev: 0.1987 sec, CV: 0.35%


----------------------------------

**------KD Hybrid Log Run 1--------**

In [13]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.4781 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 758.05it/s]


Total Search Time: 0.053949 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 97828.61it/s]


Insert Time: 0.1043 sec


...

Total Search Time: 0.940347 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 68356.79it/s]


Insert Time: 0.1486 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 90.21it/s]

Total Search Time: 1.948800 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.4781 sec, Total Search Time: 0.053949 sec, Total Time: 0.5320 sec
Points: 110,000, Insert Time: 0.1043 sec, Total Search Time: 0.172898 sec, Total Time: 0.2772 sec
Points: 210,000, Insert Time: 0.9052 sec, Total Search Time: 0.297568 sec, Total Time: 1.2027 sec
Points: 220,000, Insert Time: 0.0916 sec, Total Search Time: 0.123389 sec, Total Time: 0.2149 sec
Points: 320,000, Insert Time: 0.8321 sec, Total Search Time: 0.181365 sec, Total Time: 1.0135 sec
Points: 1,320,000, Build Time: 12.3109 sec, Total Search Time: 5.399475 sec, Total Time: 17.7104 sec
Points: 1,330,000, Insert Time: 0.1601 sec, Total Search Time: 5.521152 sec, Total Time: 5.6812 sec
Points: 1,340,000, Insert Time: 0.1614 sec, Total Search Time: 0.513814 sec, Total Time: 0.6752 sec
Points: 1,440,000, Insert Time: 1.1730 sec, Total Search Time: 1.341958 sec, Total Time: 2.5150 sec
Points: 1,540,000, Insert Time: 1.070

**-------KD Hybrid Log Run 2-----------**

In [18]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 1.0812 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 763.36it/s]


Total Search Time: 0.057851 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 132767.27it/s]


Insert Time: 0.0779 sec


...

Total Search Time: 0.929326 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 104068.24it/s]


Insert Time: 0.0986 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 88.89it/s]

Total Search Time: 1.875957 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 1.0812 sec, Total Search Time: 0.057851 sec, Total Time: 1.1390 sec
Points: 110,000, Insert Time: 0.0779 sec, Total Search Time: 0.165765 sec, Total Time: 0.2436 sec
Points: 210,000, Insert Time: 0.9149 sec, Total Search Time: 0.290964 sec, Total Time: 1.2059 sec
Points: 220,000, Insert Time: 0.1697 sec, Total Search Time: 0.133968 sec, Total Time: 0.3037 sec
Points: 320,000, Insert Time: 0.9360 sec, Total Search Time: 0.224770 sec, Total Time: 1.1608 sec
Points: 1,320,000, Build Time: 11.8050 sec, Total Search Time: 5.408500 sec, Total Time: 17.2135 sec
Points: 1,330,000, Insert Time: 0.1329 sec, Total Search Time: 5.464875 sec, Total Time: 5.5977 sec
Points: 1,340,000, Insert Time: 0.1088 sec, Total Search Time: 0.547268 sec, Total Time: 0.6560 sec
Points: 1,440,000, Insert Time: 1.0130 sec, Total Search Time: 1.390600 sec, Total Time: 2.4036 sec
Points: 1,540,000, Insert Time: 1.151

**------KD Hybrid Log Run 3------**

In [23]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.4755 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 851.29it/s]


Total Search Time: 0.051226 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123684.55it/s]


Insert Time: 0.0830 sec


...

Total Search Time: 0.994545 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 107005.74it/s]


Insert Time: 0.0955 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 93.62it/s]

Total Search Time: 1.896425 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.4755 sec, Total Search Time: 0.051226 sec, Total Time: 0.5268 sec
Points: 110,000, Insert Time: 0.0830 sec, Total Search Time: 0.161330 sec, Total Time: 0.2443 sec
Points: 210,000, Insert Time: 0.8796 sec, Total Search Time: 0.277769 sec, Total Time: 1.1574 sec
Points: 220,000, Insert Time: 0.0780 sec, Total Search Time: 0.152754 sec, Total Time: 0.2308 sec
Points: 320,000, Insert Time: 0.8420 sec, Total Search Time: 0.230136 sec, Total Time: 1.0721 sec
Points: 1,320,000, Build Time: 12.5835 sec, Total Search Time: 5.469414 sec, Total Time: 18.0529 sec
Points: 1,330,000, Insert Time: 0.1298 sec, Total Search Time: 5.509265 sec, Total Time: 5.6391 sec
Points: 1,340,000, Insert Time: 0.1395 sec, Total Search Time: 0.518361 sec, Total Time: 0.6578 sec
Points: 1,440,000, Insert Time: 1.0487 sec, Total Search Time: 1.401679 sec, Total Time: 2.4504 sec
Points: 1,540,000, Insert Time: 1.000

**---------KD Hybrid Log Analysis------------**

In [26]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 56.6280',
    'Total Time:: 56.4212',
    'Total Time:: 56.9054'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 56.6515 sec, Std Dev: 0.2430 sec, CV: 0.43%


-----------------------------------

**--------KD Hybrid Log Ratio Run 1---------**

In [14]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.4733 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 768.75it/s]


Total Search Time: 0.053188 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 120056.79it/s]


Insert Time: 0.0855 sec


...

Total Search Time: 1.024763 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 103232.21it/s]


Insert Time: 0.0992 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 89.40it/s]

Total Search Time: 1.892489 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.4733 sec, Total Search Time: 0.053188 sec, Total Time: 0.5265 sec
Points: 110,000, Insert Time: 0.0855 sec, Total Search Time: 0.170208 sec, Total Time: 0.2557 sec
Points: 210,000, Insert Time: 0.8560 sec, Total Search Time: 0.266298 sec, Total Time: 1.1223 sec
Points: 220,000, Insert Time: 0.0929 sec, Total Search Time: 0.148904 sec, Total Time: 0.2418 sec
Points: 320,000, Insert Time: 0.8452 sec, Total Search Time: 0.228358 sec, Total Time: 1.0736 sec
Points: 1,320,000, Build Time: 11.9363 sec, Total Search Time: 5.382736 sec, Total Time: 17.3190 sec
Points: 1,330,000, Insert Time: 0.1236 sec, Total Search Time: 5.618757 sec, Total Time: 5.7423 sec
Points: 1,340,000, Insert Time: 0.1109 sec, Total Search Time: 0.498409 sec, Total Time: 0.6093 sec
Points: 1,440,000, Insert Time: 1.1123 sec, Total Search Time: 1.354569 sec, Total Time: 2.4669 sec
Points: 1,540,000, Insert Time: 1.092

**------KD Hybrid Log Ratio Run 2---------**

In [19]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 1.0426 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 787.15it/s]


Total Search Time: 0.058749 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 134703.09it/s]


Insert Time: 0.0777 sec


...

Total Search Time: 0.983083 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 89200.84it/s]


Insert Time: 0.1143 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 88.54it/s]

Total Search Time: 1.919175 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 1.0426 sec, Total Search Time: 0.058749 sec, Total Time: 1.1013 sec
Points: 110,000, Insert Time: 0.0777 sec, Total Search Time: 0.171235 sec, Total Time: 0.2490 sec
Points: 210,000, Insert Time: 0.8520 sec, Total Search Time: 0.291594 sec, Total Time: 1.1436 sec
Points: 220,000, Insert Time: 0.0874 sec, Total Search Time: 0.153746 sec, Total Time: 0.2411 sec
Points: 320,000, Insert Time: 0.8801 sec, Total Search Time: 0.210827 sec, Total Time: 1.0909 sec
Points: 1,320,000, Build Time: 11.8688 sec, Total Search Time: 5.415828 sec, Total Time: 17.2846 sec
Points: 1,330,000, Insert Time: 0.1472 sec, Total Search Time: 5.448656 sec, Total Time: 5.5959 sec
Points: 1,340,000, Insert Time: 0.0957 sec, Total Search Time: 0.537693 sec, Total Time: 0.6334 sec
Points: 1,440,000, Insert Time: 1.1269 sec, Total Search Time: 1.394065 sec, Total Time: 2.5209 sec
Points: 1,540,000, Insert Time: 1.053

**------KD Hybrid Log Ratio Run 3-----**

In [24]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2250
epsilon_rebuild = 0.099

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.4985 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 894.62it/s]


Total Search Time: 0.051989 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 132245.26it/s]


Insert Time: 0.0794 sec


...

Total Search Time: 0.996810 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 105452.62it/s]


Insert Time: 0.0978 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 95.60it/s]

Total Search Time: 1.867246 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.4985 sec, Total Search Time: 0.051989 sec, Total Time: 0.5505 sec
Points: 110,000, Insert Time: 0.0794 sec, Total Search Time: 0.165095 sec, Total Time: 0.2445 sec
Points: 210,000, Insert Time: 0.9227 sec, Total Search Time: 0.275619 sec, Total Time: 1.1983 sec
Points: 220,000, Insert Time: 0.0824 sec, Total Search Time: 0.144248 sec, Total Time: 0.2267 sec
Points: 320,000, Insert Time: 0.7683 sec, Total Search Time: 0.192000 sec, Total Time: 0.9603 sec
Points: 1,320,000, Build Time: 12.2717 sec, Total Search Time: 5.393138 sec, Total Time: 17.6649 sec
Points: 1,330,000, Insert Time: 0.1309 sec, Total Search Time: 5.548110 sec, Total Time: 5.6790 sec
Points: 1,340,000, Insert Time: 0.1312 sec, Total Search Time: 0.519137 sec, Total Time: 0.6504 sec
Points: 1,440,000, Insert Time: 1.0804 sec, Total Search Time: 1.392341 sec, Total Time: 2.4728 sec
Points: 1,540,000, Insert Time: 1.085

**---------KD Hybrid Log Ratio Analysis----------**

In [25]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 55.6198',
    'Total Time:: 56.7367',
    'Total Time:: 56.1922'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 56.1829 sec, Std Dev: 0.5585 sec, CV: 0.99%


------------------------------------

-----------------------------------

**----------KD Dynamic---------**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

-----------------

**---------KD Dyn Sqrt Run 1--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.6884 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 851.07it/s]


Total Search Time: 0.058663 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 88058.47it/s]

Insert Time: 0.1158 sec


...

Total Search Time: 1.029780 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 84791.83it/s]


Insert Time: 0.1201 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 104.14it/s]

Total Search Time: 1.803103 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.6884 sec, Total Search Time: 0.058663 sec, Total Time: 0.7471 sec
Points: 110,000, Insert Time: 0.1158 sec, Total Search Time: 0.167019 sec, Total Time: 0.2828 sec
Points: 210,000, Build Time: 1.6235 sec, Total Search Time: 0.262256 sec, Total Time: 1.8858 sec
Points: 220,000, Insert Time: 0.1073 sec, Total Search Time: 0.117557 sec, Total Time: 0.2249 sec
Points: 320,000, Insert Time: 0.9990 sec, Total Search Time: 0.167701 sec, Total Time: 1.1667 sec
Points: 1,320,000, Build Time: 13.2749 sec, Total Search Time: 4.683462 sec, Total Time: 17.9584 sec
Points: 1,330,000, Insert Time: 0.1208 sec, Total Search Time: 4.865419 sec, Total Time: 4.9862 sec
Points: 1,340,000, Insert Time: 0.0985 sec, Total Search Time: 0.523742 sec, Total Time: 0.6223 sec
Points: 1,440,000, Insert Time: 1.1752 sec, Total Search Time: 1.286046 sec, Total Time: 2.4612 sec
Points: 1,540,000, Insert Time: 1.2308

**------KD Dyn Sqrt Run 2--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5335 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 800.97it/s]


Total Search Time: 0.057368 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 125061.62it/s]


Insert Time: 0.0823 sec


...

Total Search Time: 0.938778 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 76104.40it/s]


Insert Time: 0.1337 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 97.94it/s]

Total Search Time: 1.831490 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5335 sec, Total Search Time: 0.057368 sec, Total Time: 0.5908 sec
Points: 110,000, Insert Time: 0.0823 sec, Total Search Time: 0.156762 sec, Total Time: 0.2391 sec
Points: 210,000, Build Time: 2.2077 sec, Total Search Time: 0.254214 sec, Total Time: 2.4619 sec
Points: 220,000, Insert Time: 0.1233 sec, Total Search Time: 0.113480 sec, Total Time: 0.2368 sec
Points: 320,000, Insert Time: 0.9301 sec, Total Search Time: 0.168583 sec, Total Time: 1.0987 sec
Points: 1,320,000, Build Time: 13.3927 sec, Total Search Time: 4.720368 sec, Total Time: 18.1130 sec
Points: 1,330,000, Insert Time: 0.1275 sec, Total Search Time: 4.860873 sec, Total Time: 4.9884 sec
Points: 1,340,000, Insert Time: 0.1283 sec, Total Search Time: 0.519130 sec, Total Time: 0.6475 sec
Points: 1,440,000, Insert Time: 1.1352 sec, Total Search Time: 1.326550 sec, Total Time: 2.4618 sec
Points: 1,540,000, Insert Time: 1.3218

**-----KD Dyn Sqrt Run 3--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5067 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 962.05it/s]


Total Search Time: 0.050458 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 131468.87it/s]


Insert Time: 0.0782 sec


...

Total Search Time: 0.990126 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 81470.32it/s]


Insert Time: 0.1254 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 110.63it/s]

Total Search Time: 1.766049 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5067 sec, Total Search Time: 0.050458 sec, Total Time: 0.5571 sec
Points: 110,000, Insert Time: 0.0782 sec, Total Search Time: 0.156021 sec, Total Time: 0.2342 sec
Points: 210,000, Build Time: 2.2674 sec, Total Search Time: 0.275803 sec, Total Time: 2.5432 sec
Points: 220,000, Insert Time: 0.0997 sec, Total Search Time: 0.129239 sec, Total Time: 0.2289 sec
Points: 320,000, Insert Time: 0.9009 sec, Total Search Time: 0.179215 sec, Total Time: 1.0801 sec
Points: 1,320,000, Build Time: 13.7612 sec, Total Search Time: 4.666700 sec, Total Time: 18.4279 sec
Points: 1,330,000, Insert Time: 0.1135 sec, Total Search Time: 4.809453 sec, Total Time: 4.9230 sec
Points: 1,340,000, Insert Time: 0.1338 sec, Total Search Time: 0.544071 sec, Total Time: 0.6779 sec
Points: 1,440,000, Insert Time: 1.1610 sec, Total Search Time: 1.293360 sec, Total Time: 2.4543 sec
Points: 1,540,000, Insert Time: 1.3283

**------KD Dyn Sqrt Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 57.1081',
    'Total Time:: 57.1769',
    'Total Time:: 57.7665'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.3505 sec, Std Dev: 0.3619 sec, CV: 0.63%


---------------------

**-------KD Dyn Log Run 1-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 1.1236 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 856.57it/s]


Total Search Time: 0.056296 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 128887.36it/s]


Insert Time: 0.0798 sec


...

Total Search Time: 1.017871 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 78626.44it/s]


Insert Time: 0.1300 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 105.49it/s]

Total Search Time: 1.815199 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 1.1236 sec, Total Search Time: 0.056296 sec, Total Time: 1.1799 sec
Points: 110,000, Insert Time: 0.0798 sec, Total Search Time: 0.162125 sec, Total Time: 0.2419 sec
Points: 210,000, Insert Time: 0.8629 sec, Total Search Time: 0.290861 sec, Total Time: 1.1538 sec
Points: 220,000, Insert Time: 0.1113 sec, Total Search Time: 0.150918 sec, Total Time: 0.2622 sec
Points: 320,000, Insert Time: 0.8821 sec, Total Search Time: 0.215907 sec, Total Time: 1.0980 sec
Points: 1,320,000, Build Time: 13.3392 sec, Total Search Time: 4.669230 sec, Total Time: 18.0084 sec
Points: 1,330,000, Insert Time: 0.1381 sec, Total Search Time: 4.807167 sec, Total Time: 4.9452 sec
Points: 1,340,000, Insert Time: 0.1390 sec, Total Search Time: 0.523346 sec, Total Time: 0.6624 sec
Points: 1,440,000, Insert Time: 1.1289 sec, Total Search Time: 1.268399 sec, Total Time: 2.3973 sec
Points: 1,540,000, Insert Time: 1.221

**-----KD Dyn Log Run 2--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5366 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 849.73it/s]


Total Search Time: 0.054549 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 90979.97it/s]


Insert Time: 0.1122 sec


...

Total Search Time: 0.957901 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 72556.15it/s]


Insert Time: 0.1415 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 96.03it/s]

Total Search Time: 1.831123 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5366 sec, Total Search Time: 0.054549 sec, Total Time: 0.5911 sec
Points: 110,000, Insert Time: 0.1122 sec, Total Search Time: 0.162150 sec, Total Time: 0.2744 sec
Points: 210,000, Insert Time: 0.8557 sec, Total Search Time: 0.300259 sec, Total Time: 1.1560 sec
Points: 220,000, Insert Time: 0.1075 sec, Total Search Time: 0.132150 sec, Total Time: 0.2396 sec
Points: 320,000, Insert Time: 0.9365 sec, Total Search Time: 0.228140 sec, Total Time: 1.1646 sec
Points: 1,320,000, Build Time: 14.1162 sec, Total Search Time: 4.691726 sec, Total Time: 18.8079 sec
Points: 1,330,000, Insert Time: 0.1588 sec, Total Search Time: 4.832018 sec, Total Time: 4.9908 sec
Points: 1,340,000, Insert Time: 0.1397 sec, Total Search Time: 0.526005 sec, Total Time: 0.6657 sec
Points: 1,440,000, Insert Time: 1.2336 sec, Total Search Time: 1.289192 sec, Total Time: 2.5228 sec
Points: 1,540,000, Insert Time: 1.234

**-----KD Dyn Log Run 3------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5149 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 957.32it/s]


Total Search Time: 0.050979 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 64111.03it/s]


Insert Time: 0.1607 sec


...

Total Search Time: 1.003736 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 93006.28it/s]


Insert Time: 0.1111 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 110.22it/s]

Total Search Time: 1.742747 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5149 sec, Total Search Time: 0.050979 sec, Total Time: 0.5659 sec
Points: 110,000, Insert Time: 0.1607 sec, Total Search Time: 0.156417 sec, Total Time: 0.3171 sec
Points: 210,000, Insert Time: 0.9501 sec, Total Search Time: 0.273569 sec, Total Time: 1.2237 sec
Points: 220,000, Insert Time: 0.0865 sec, Total Search Time: 0.139762 sec, Total Time: 0.2262 sec
Points: 320,000, Insert Time: 0.9614 sec, Total Search Time: 0.242516 sec, Total Time: 1.2040 sec
Points: 1,320,000, Build Time: 14.1163 sec, Total Search Time: 4.632033 sec, Total Time: 18.7483 sec
Points: 1,330,000, Insert Time: 0.1246 sec, Total Search Time: 4.805911 sec, Total Time: 4.9305 sec
Points: 1,340,000, Insert Time: 0.1573 sec, Total Search Time: 0.534904 sec, Total Time: 0.6922 sec
Points: 1,440,000, Insert Time: 1.3231 sec, Total Search Time: 1.293554 sec, Total Time: 2.6166 sec
Points: 1,540,000, Insert Time: 1.158

**------KD Dyn Log Analysis-------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 56.3864',
    'Total Time:: 57.0674',
    'Total Time:: 56.9209'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 56.7916 sec, Std Dev: 0.3584 sec, CV: 0.63%


-------------------

**------KD Tree Dynamic Log Ratio Run 1--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5240 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 868.77it/s]


Total Search Time: 0.054882 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 121119.39it/s]


Insert Time: 0.0860 sec


...

Total Search Time: 0.982680 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 85971.05it/s]


Insert Time: 0.1192 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 104.27it/s]

Total Search Time: 1.817214 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5240 sec, Total Search Time: 0.054882 sec, Total Time: 0.5789 sec
Points: 110,000, Insert Time: 0.0860 sec, Total Search Time: 0.167581 sec, Total Time: 0.2536 sec
Points: 210,000, Insert Time: 0.9193 sec, Total Search Time: 0.304775 sec, Total Time: 1.2241 sec
Points: 220,000, Insert Time: 0.1170 sec, Total Search Time: 0.147095 sec, Total Time: 0.2641 sec
Points: 320,000, Insert Time: 1.0166 sec, Total Search Time: 0.212690 sec, Total Time: 1.2293 sec
Points: 1,320,000, Build Time: 14.3296 sec, Total Search Time: 4.675435 sec, Total Time: 19.0050 sec
Points: 1,330,000, Insert Time: 0.1325 sec, Total Search Time: 4.825628 sec, Total Time: 4.9581 sec
Points: 1,340,000, Insert Time: 0.0957 sec, Total Search Time: 0.511456 sec, Total Time: 0.6071 sec
Points: 1,440,000, Insert Time: 1.1930 sec, Total Search Time: 1.289056 sec, Total Time: 2.4821 sec
Points: 1,540,000, Insert Time: 1.169

**--------KD Dyn Log Ratio Run 2------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5376 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 836.91it/s]


Total Search Time: 0.053908 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 78399.52it/s]

Insert Time: 0.1294 sec


...

Total Search Time: 0.979116 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 96447.61it/s]


Insert Time: 0.1063 sec


Querying batch 20: 100%|██████████| 100/100 [00:01<00:00, 97.94it/s]

Total Search Time: 1.867491 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5376 sec, Total Search Time: 0.053908 sec, Total Time: 0.5916 sec
Points: 110,000, Insert Time: 0.1294 sec, Total Search Time: 0.160846 sec, Total Time: 0.2903 sec
Points: 210,000, Insert Time: 0.8968 sec, Total Search Time: 0.298493 sec, Total Time: 1.1953 sec
Points: 220,000, Insert Time: 0.0814 sec, Total Search Time: 0.146523 sec, Total Time: 0.2279 sec
Points: 320,000, Insert Time: 0.8808 sec, Total Search Time: 0.219398 sec, Total Time: 1.1002 sec
Points: 1,320,000, Build Time: 14.2650 sec, Total Search Time: 4.697421 sec, Total Time: 18.9624 sec
Points: 1,330,000, Insert Time: 0.1073 sec, Total Search Time: 4.829397 sec, Total Time: 4.9367 sec
Points: 1,340,000, Insert Time: 0.1203 sec, Total Search Time: 0.530371 sec, Total Time: 0.6507 sec
Points: 1,440,000, Insert Time: 1.2292 sec, Total Search Time: 1.285656 sec, Total Time: 2.5149 sec
Points: 1,540,000, Insert Time: 1.245

**-------KD Dynamic Log Ratio Run 3----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2000
epsilon_rebuild = 0.068

epsilon_insert = 0.068

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.7005 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 960.86it/s]


Total Search Time: 0.051593 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 105423.99it/s]


Insert Time: 0.0975 sec


...

Total Search Time: 1.070585 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 79332.40it/s]


Insert Time: 0.1284 sec


Querying batch 20: 100%|██████████| 100/100 [00:00<00:00, 106.84it/s]

Total Search Time: 1.757265 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.7005 sec, Total Search Time: 0.051593 sec, Total Time: 0.7521 sec
Points: 110,000, Insert Time: 0.0975 sec, Total Search Time: 0.155354 sec, Total Time: 0.2528 sec
Points: 210,000, Insert Time: 0.9738 sec, Total Search Time: 0.284142 sec, Total Time: 1.2580 sec
Points: 220,000, Insert Time: 0.0922 sec, Total Search Time: 0.148596 sec, Total Time: 0.2408 sec
Points: 320,000, Insert Time: 1.0002 sec, Total Search Time: 0.214516 sec, Total Time: 1.2148 sec
Points: 1,320,000, Build Time: 14.6198 sec, Total Search Time: 4.757168 sec, Total Time: 19.3769 sec
Points: 1,330,000, Insert Time: 0.1070 sec, Total Search Time: 4.817303 sec, Total Time: 4.9243 sec
Points: 1,340,000, Insert Time: 0.1258 sec, Total Search Time: 0.539293 sec, Total Time: 0.6651 sec
Points: 1,440,000, Insert Time: 1.0948 sec, Total Search Time: 1.307872 sec, Total Time: 2.4027 sec
Points: 1,540,000, Insert Time: 1.223

**-------KD Dyn Log Ratio Analysis-------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 57.2085',
    'Total Time:: 56.7122',
    'Total Time:: 57.5761'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.1656 sec, Std Dev: 0.4335 sec, CV: 0.76%


------------------

--------------

**----------VP Tree Hybrid------**

In [ ]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

-------------

**-----VP Hybrid Sqrt Run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1074 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 10393.52it/s]


Total Search Time: 0.003852 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 119795.50it/s]


Insert Time: 0.0856 sec


...

Total Search Time: 0.966273 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 120353.40it/s]


Insert Time: 0.0851 sec


Querying batch 20: 100%|██████████| 100/100 [00:04<00:00, 24.19it/s]

Total Search Time: 4.283178 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1074 sec, Total Search Time: 0.003852 sec, Total Time: 0.1113 sec
Points: 110,000, Insert Time: 0.0856 sec, Total Search Time: 0.045279 sec, Total Time: 0.1309 sec
Points: 210,000, Build Time: 0.4342 sec, Total Search Time: 0.010248 sec, Total Time: 0.4444 sec
Points: 220,000, Insert Time: 0.0840 sec, Total Search Time: 0.014183 sec, Total Time: 0.0982 sec
Points: 320,000, Insert Time: 0.8283 sec, Total Search Time: 0.107281 sec, Total Time: 0.9355 sec
Points: 1,320,000, Build Time: 2.1760 sec, Total Search Time: 0.128182 sec, Total Time: 2.3042 sec
Points: 1,330,000, Insert Time: 0.0869 sec, Total Search Time: 0.293633 sec, Total Time: 0.3806 sec
Points: 1,340,000, Insert Time: 0.0976 sec, Total Search Time: 0.026624 sec, Total Time: 0.1242 sec
Points: 1,440,000, Insert Time: 0.8538 sec, Total Search Time: 0.522791 sec, Total Time: 1.3766 sec
Points: 1,540,000, Insert Time: 0.8402 s

**------VP Hybrid Sqrt Run 2-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1541 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 18645.49it/s]


Total Search Time: 0.003443 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 101677.88it/s]


Insert Time: 0.1004 sec


...

Total Search Time: 0.968356 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 117920.21it/s]


Insert Time: 0.0880 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 26.20it/s]

Total Search Time: 3.967636 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1541 sec, Total Search Time: 0.003443 sec, Total Time: 0.1576 sec
Points: 110,000, Insert Time: 0.1004 sec, Total Search Time: 0.046503 sec, Total Time: 0.1469 sec
Points: 210,000, Build Time: 0.2469 sec, Total Search Time: 0.006315 sec, Total Time: 0.2532 sec
Points: 220,000, Insert Time: 0.0829 sec, Total Search Time: 0.014033 sec, Total Time: 0.0970 sec
Points: 320,000, Insert Time: 0.8474 sec, Total Search Time: 0.105405 sec, Total Time: 0.9528 sec
Points: 1,320,000, Build Time: 2.3639 sec, Total Search Time: 0.027964 sec, Total Time: 2.3919 sec
Points: 1,330,000, Insert Time: 0.0831 sec, Total Search Time: 0.211142 sec, Total Time: 0.2942 sec
Points: 1,340,000, Insert Time: 0.0870 sec, Total Search Time: 0.023195 sec, Total Time: 0.1102 sec
Points: 1,440,000, Insert Time: 0.8163 sec, Total Search Time: 0.493446 sec, Total Time: 1.3097 sec
Points: 1,540,000, Insert Time: 0.8461 s

**------VP Hybrid Sqrt Run 3--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1419 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 25536.10it/s]


Total Search Time: 0.003815 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123786.76it/s]


Insert Time: 0.0837 sec


...

Total Search Time: 0.977312 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 117921.21it/s]


Insert Time: 0.0873 sec


Querying batch 20: 100%|██████████| 100/100 [00:04<00:00, 24.18it/s]

Total Search Time: 4.283185 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1419 sec, Total Search Time: 0.003815 sec, Total Time: 0.1457 sec
Points: 110,000, Insert Time: 0.0837 sec, Total Search Time: 0.046471 sec, Total Time: 0.1301 sec
Points: 210,000, Build Time: 0.2597 sec, Total Search Time: 0.007157 sec, Total Time: 0.2668 sec
Points: 220,000, Insert Time: 0.0864 sec, Total Search Time: 0.013621 sec, Total Time: 0.1001 sec
Points: 320,000, Insert Time: 0.8169 sec, Total Search Time: 0.109310 sec, Total Time: 0.9263 sec
Points: 1,320,000, Build Time: 2.3890 sec, Total Search Time: 0.027106 sec, Total Time: 2.4161 sec
Points: 1,330,000, Insert Time: 0.0946 sec, Total Search Time: 0.217008 sec, Total Time: 0.3116 sec
Points: 1,340,000, Insert Time: 0.0853 sec, Total Search Time: 0.022881 sec, Total Time: 0.1081 sec
Points: 1,440,000, Insert Time: 0.8561 sec, Total Search Time: 0.499319 sec, Total Time: 1.3555 sec
Points: 1,540,000, Insert Time: 0.8179 s

**------VP Hybrid Sqrt Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 46.2344',
    'Total Time:: 41.6938',
    'Total Time:: 42.3866'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 43.4383 sec, Std Dev: 2.4462 sec, CV: 5.63%


-----------------

**-----VP Hybrid Log Run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1458 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 11279.56it/s]


Total Search Time: 0.004050 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 122277.92it/s]


Insert Time: 0.0843 sec


...

Total Search Time: 0.964003 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 116234.00it/s]


Insert Time: 0.0882 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 25.93it/s]

Total Search Time: 4.007536 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1458 sec, Total Search Time: 0.004050 sec, Total Time: 0.1498 sec
Points: 110,000, Insert Time: 0.0843 sec, Total Search Time: 0.046648 sec, Total Time: 0.1310 sec
Points: 210,000, Insert Time: 0.8396 sec, Total Search Time: 0.461713 sec, Total Time: 1.3013 sec
Points: 220,000, Insert Time: 0.0907 sec, Total Search Time: 0.117097 sec, Total Time: 0.2078 sec
Points: 320,000, Insert Time: 0.8098 sec, Total Search Time: 0.218842 sec, Total Time: 1.0286 sec
Points: 1,320,000, Build Time: 2.3182 sec, Total Search Time: 0.126614 sec, Total Time: 2.4448 sec
Points: 1,330,000, Insert Time: 0.0832 sec, Total Search Time: 0.271617 sec, Total Time: 0.3548 sec
Points: 1,340,000, Insert Time: 0.0849 sec, Total Search Time: 0.027237 sec, Total Time: 0.1122 sec
Points: 1,440,000, Insert Time: 0.8333 sec, Total Search Time: 0.516518 sec, Total Time: 1.3498 sec
Points: 1,540,000, Insert Time: 0.8270 

**-------VP Hybrid Log Run 2-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1234 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 13484.34it/s]


Total Search Time: 0.003375 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 85860.35it/s]


Insert Time: 0.1195 sec


...

Total Search Time: 0.956461 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123600.72it/s]


Insert Time: 0.0828 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 26.36it/s]

Total Search Time: 3.944930 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1234 sec, Total Search Time: 0.003375 sec, Total Time: 0.1268 sec
Points: 110,000, Insert Time: 0.1195 sec, Total Search Time: 0.045530 sec, Total Time: 0.1650 sec
Points: 210,000, Insert Time: 0.8364 sec, Total Search Time: 0.454536 sec, Total Time: 1.2909 sec
Points: 220,000, Insert Time: 0.0841 sec, Total Search Time: 0.120191 sec, Total Time: 0.2043 sec
Points: 320,000, Insert Time: 0.8350 sec, Total Search Time: 0.216901 sec, Total Time: 1.0519 sec
Points: 1,320,000, Build Time: 2.3660 sec, Total Search Time: 0.027173 sec, Total Time: 2.3932 sec
Points: 1,330,000, Insert Time: 0.0838 sec, Total Search Time: 0.218814 sec, Total Time: 0.3026 sec
Points: 1,340,000, Insert Time: 0.0873 sec, Total Search Time: 0.023222 sec, Total Time: 0.1105 sec
Points: 1,440,000, Insert Time: 0.8264 sec, Total Search Time: 0.498117 sec, Total Time: 1.3246 sec
Points: 1,540,000, Insert Time: 0.8200 

**-----VP Hybrid Log Run 3--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1576 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 15001.09it/s]


Total Search Time: 0.003794 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123484.64it/s]


Insert Time: 0.0843 sec


...

Total Search Time: 0.996493 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 122407.82it/s]


Insert Time: 0.0842 sec


Querying batch 20: 100%|██████████| 100/100 [00:04<00:00, 24.27it/s]

Total Search Time: 4.273781 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1576 sec, Total Search Time: 0.003794 sec, Total Time: 0.1614 sec
Points: 110,000, Insert Time: 0.0843 sec, Total Search Time: 0.047213 sec, Total Time: 0.1315 sec
Points: 210,000, Insert Time: 0.8313 sec, Total Search Time: 0.470299 sec, Total Time: 1.3016 sec
Points: 220,000, Insert Time: 0.0894 sec, Total Search Time: 0.116788 sec, Total Time: 0.2062 sec
Points: 320,000, Insert Time: 0.8307 sec, Total Search Time: 0.216475 sec, Total Time: 1.0472 sec
Points: 1,320,000, Build Time: 2.3505 sec, Total Search Time: 0.027196 sec, Total Time: 2.3777 sec
Points: 1,330,000, Insert Time: 0.0852 sec, Total Search Time: 0.217879 sec, Total Time: 0.3031 sec
Points: 1,340,000, Insert Time: 0.0876 sec, Total Search Time: 0.024080 sec, Total Time: 0.1117 sec
Points: 1,440,000, Insert Time: 0.8384 sec, Total Search Time: 0.492677 sec, Total Time: 1.3311 sec
Points: 1,540,000, Insert Time: 0.8309 

**-----VP Hybrid Log Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 43.2611',
    'Total Time:: 42.9598',
    'Total Time:: 43.7474'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 43.3228 sec, Std Dev: 0.3974 sec, CV: 0.92%


------------

**-----VP Hybrid Log Ratio Run 1-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1546 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 11602.50it/s]


Total Search Time: 0.004303 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 121527.53it/s]


Insert Time: 0.0846 sec


...

Total Search Time: 0.963022 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123112.04it/s]


Insert Time: 0.0847 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 26.04it/s]

Total Search Time: 3.989274 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1546 sec, Total Search Time: 0.004303 sec, Total Time: 0.1589 sec
Points: 110,000, Insert Time: 0.0846 sec, Total Search Time: 0.046111 sec, Total Time: 0.1307 sec
Points: 210,000, Insert Time: 0.8190 sec, Total Search Time: 0.449421 sec, Total Time: 1.2684 sec
Points: 220,000, Insert Time: 0.0845 sec, Total Search Time: 0.118536 sec, Total Time: 0.2030 sec
Points: 320,000, Insert Time: 0.8246 sec, Total Search Time: 0.216949 sec, Total Time: 1.0416 sec
Points: 1,320,000, Build Time: 2.3406 sec, Total Search Time: 0.127343 sec, Total Time: 2.4680 sec
Points: 1,330,000, Insert Time: 0.0841 sec, Total Search Time: 0.268821 sec, Total Time: 0.3529 sec
Points: 1,340,000, Insert Time: 0.0850 sec, Total Search Time: 0.026565 sec, Total Time: 0.1115 sec
Points: 1,440,000, Insert Time: 0.8620 sec, Total Search Time: 0.543723 sec, Total Time: 1.4058 sec
Points: 1,540,000, Insert Time: 0.8618 

**------VP Hybrid Log Ratio Run 2---------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1415 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 13682.28it/s]


Total Search Time: 0.003329 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 95635.91it/s]


Insert Time: 0.1066 sec


...

Total Search Time: 0.969458 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 111646.25it/s]


Insert Time: 0.0919 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 26.07it/s]

Total Search Time: 3.998921 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1415 sec, Total Search Time: 0.003329 sec, Total Time: 0.1448 sec
Points: 110,000, Insert Time: 0.1066 sec, Total Search Time: 0.045778 sec, Total Time: 0.1524 sec
Points: 210,000, Insert Time: 0.8279 sec, Total Search Time: 0.462943 sec, Total Time: 1.2908 sec
Points: 220,000, Insert Time: 0.0872 sec, Total Search Time: 0.113764 sec, Total Time: 0.2009 sec
Points: 320,000, Insert Time: 0.8182 sec, Total Search Time: 0.219872 sec, Total Time: 1.0381 sec
Points: 1,320,000, Build Time: 2.3238 sec, Total Search Time: 0.032738 sec, Total Time: 2.3566 sec
Points: 1,330,000, Insert Time: 0.0860 sec, Total Search Time: 0.214106 sec, Total Time: 0.3001 sec
Points: 1,340,000, Insert Time: 0.1015 sec, Total Search Time: 0.022894 sec, Total Time: 0.1244 sec
Points: 1,440,000, Insert Time: 0.8418 sec, Total Search Time: 0.503279 sec, Total Time: 1.3451 sec
Points: 1,540,000, Insert Time: 0.8350 

**------VP Hybrid Log Ratio Run 3----------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700
epsilon_insert = 13.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1369 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 21664.79it/s]


Total Search Time: 0.003258 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 124396.94it/s]


Insert Time: 0.0836 sec


...

Total Search Time: 0.972506 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 118571.93it/s]


Insert Time: 0.0863 sec


Querying batch 20: 100%|██████████| 100/100 [00:04<00:00, 24.30it/s]

Total Search Time: 4.265963 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1369 sec, Total Search Time: 0.003258 sec, Total Time: 0.1401 sec
Points: 110,000, Insert Time: 0.0836 sec, Total Search Time: 0.043566 sec, Total Time: 0.1272 sec
Points: 210,000, Insert Time: 0.8249 sec, Total Search Time: 0.450168 sec, Total Time: 1.2751 sec
Points: 220,000, Insert Time: 0.0832 sec, Total Search Time: 0.115714 sec, Total Time: 0.1989 sec
Points: 320,000, Insert Time: 0.9049 sec, Total Search Time: 0.213056 sec, Total Time: 1.1179 sec
Points: 1,320,000, Build Time: 2.3432 sec, Total Search Time: 0.027040 sec, Total Time: 2.3702 sec
Points: 1,330,000, Insert Time: 0.0835 sec, Total Search Time: 0.216033 sec, Total Time: 0.2996 sec
Points: 1,340,000, Insert Time: 0.0849 sec, Total Search Time: 0.023440 sec, Total Time: 0.1083 sec
Points: 1,440,000, Insert Time: 0.8323 sec, Total Search Time: 0.513188 sec, Total Time: 1.3454 sec
Points: 1,540,000, Insert Time: 0.8354 

**-----VP Hybrid Log Ratio Analysis-----**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 43.3367',
    'Total Time:: 43.0507',
    'Total Time:: 43.4932'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 43.2935 sec, Std Dev: 0.2244 sec, CV: 0.52%


-----------------

------------------

**-------VP Tree Epsilon Dynamic-------**

In [ ]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

-----------------

**------VP Dyn Sqrt Run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1538 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 12975.42it/s]


Total Search Time: 0.004203 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 117921.54it/s]


Insert Time: 0.0884 sec


...

Total Search Time: 0.816776 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 122169.29it/s]


Insert Time: 0.0837 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 33.26it/s]

Total Search Time: 3.240506 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1538 sec, Total Search Time: 0.004203 sec, Total Time: 0.1580 sec
Points: 110,000, Insert Time: 0.0884 sec, Total Search Time: 0.045890 sec, Total Time: 0.1343 sec
Points: 210,000, Build Time: 0.2809 sec, Total Search Time: 0.006298 sec, Total Time: 0.2872 sec
Points: 220,000, Insert Time: 0.0944 sec, Total Search Time: 0.015940 sec, Total Time: 0.1104 sec
Points: 320,000, Insert Time: 0.8609 sec, Total Search Time: 0.096159 sec, Total Time: 0.9570 sec
Points: 1,320,000, Build Time: 2.4940 sec, Total Search Time: 0.027151 sec, Total Time: 2.5212 sec
Points: 1,330,000, Insert Time: 0.0875 sec, Total Search Time: 0.197751 sec, Total Time: 0.2852 sec
Points: 1,340,000, Insert Time: 0.0871 sec, Total Search Time: 0.022654 sec, Total Time: 0.1097 sec
Points: 1,440,000, Insert Time: 0.8972 sec, Total Search Time: 0.458306 sec, Total Time: 1.3555 sec
Points: 1,540,000, Insert Time: 0.8483 s

**------VP Dyn sqrt run 2------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.4247 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 10394.81it/s]


Total Search Time: 0.004681 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 105453.94it/s]


Insert Time: 0.0978 sec


...

Total Search Time: 0.809329 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 118817.80it/s]


Insert Time: 0.0878 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 34.02it/s]

Total Search Time: 3.158429 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.4247 sec, Total Search Time: 0.004681 sec, Total Time: 0.4294 sec
Points: 110,000, Insert Time: 0.0978 sec, Total Search Time: 0.042471 sec, Total Time: 0.1403 sec
Points: 210,000, Build Time: 0.2804 sec, Total Search Time: 0.006379 sec, Total Time: 0.2868 sec
Points: 220,000, Insert Time: 0.0844 sec, Total Search Time: 0.015360 sec, Total Time: 0.0998 sec
Points: 320,000, Insert Time: 0.8182 sec, Total Search Time: 0.106156 sec, Total Time: 0.9243 sec
Points: 1,320,000, Build Time: 2.5001 sec, Total Search Time: 0.026897 sec, Total Time: 2.5270 sec
Points: 1,330,000, Insert Time: 0.0845 sec, Total Search Time: 0.195160 sec, Total Time: 0.2797 sec
Points: 1,340,000, Insert Time: 0.0890 sec, Total Search Time: 0.021866 sec, Total Time: 0.1108 sec
Points: 1,440,000, Insert Time: 0.8759 sec, Total Search Time: 0.443779 sec, Total Time: 1.3196 sec
Points: 1,540,000, Insert Time: 0.8135 s

**------VP Dyn Sqrt Run 3------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.2359 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 9097.29it/s]


Total Search Time: 0.005901 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123058.58it/s]


Insert Time: 0.0839 sec


...

Total Search Time: 0.846952 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 97932.77it/s]


Insert Time: 0.1047 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 37.86it/s]

Total Search Time: 2.953406 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.2359 sec, Total Search Time: 0.005901 sec, Total Time: 0.2418 sec
Points: 110,000, Insert Time: 0.0839 sec, Total Search Time: 0.044621 sec, Total Time: 0.1285 sec
Points: 210,000, Build Time: 0.2870 sec, Total Search Time: 0.015400 sec, Total Time: 0.3024 sec
Points: 220,000, Insert Time: 0.0847 sec, Total Search Time: 0.015419 sec, Total Time: 0.1001 sec
Points: 320,000, Insert Time: 0.8299 sec, Total Search Time: 0.094835 sec, Total Time: 0.9247 sec
Points: 1,320,000, Build Time: 2.4752 sec, Total Search Time: 0.079131 sec, Total Time: 2.5544 sec
Points: 1,330,000, Insert Time: 0.0923 sec, Total Search Time: 0.216467 sec, Total Time: 0.3087 sec
Points: 1,340,000, Insert Time: 0.0932 sec, Total Search Time: 0.022980 sec, Total Time: 0.1162 sec
Points: 1,440,000, Insert Time: 0.9639 sec, Total Search Time: 0.396738 sec, Total Time: 1.3606 sec
Points: 1,540,000, Insert Time: 0.9628 s

**-----VP Dyn sqrt analysis------**

In [ ]:
import re
import numpy as np


results = [
    'Total Time:: 37.5943',
    'Total Time:: 37.6819',
    'Total Time:: 36.2216'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 37.1659 sec, Std Dev: 0.8190 sec, CV: 2.20%


------------

**-----VP Dynamic Log Run 1------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1317 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 20174.62it/s]


Total Search Time: 0.003669 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 56223.01it/s]


Insert Time: 0.1811 sec


...

Total Search Time: 0.805867 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 120195.78it/s]


Insert Time: 0.0863 sec


Querying batch 20: 100%|██████████| 100/100 [00:03<00:00, 33.21it/s]

Total Search Time: 3.249936 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1317 sec, Total Search Time: 0.003669 sec, Total Time: 0.1354 sec
Points: 110,000, Insert Time: 0.1811 sec, Total Search Time: 0.044065 sec, Total Time: 0.2252 sec
Points: 210,000, Insert Time: 0.8238 sec, Total Search Time: 0.407052 sec, Total Time: 1.2309 sec
Points: 220,000, Insert Time: 0.0866 sec, Total Search Time: 0.122249 sec, Total Time: 0.2089 sec
Points: 320,000, Insert Time: 0.8311 sec, Total Search Time: 0.208475 sec, Total Time: 1.0396 sec
Points: 1,320,000, Build Time: 2.5125 sec, Total Search Time: 0.028462 sec, Total Time: 2.5410 sec
Points: 1,330,000, Insert Time: 0.0883 sec, Total Search Time: 0.193575 sec, Total Time: 0.2819 sec
Points: 1,340,000, Insert Time: 0.0854 sec, Total Search Time: 0.023308 sec, Total Time: 0.1087 sec
Points: 1,440,000, Insert Time: 0.8433 sec, Total Search Time: 0.449094 sec, Total Time: 1.2924 sec
Points: 1,540,000, Insert Time: 0.8146 

**--------VP Dynamic Log Run 2--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1427 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 11342.09it/s]


Total Search Time: 0.004461 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 55347.33it/s]


Insert Time: 0.1837 sec


...

Total Search Time: 0.829304 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 118316.38it/s]


Insert Time: 0.0879 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 33.71it/s]

Total Search Time: 3.199392 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1427 sec, Total Search Time: 0.004461 sec, Total Time: 0.1472 sec
Points: 110,000, Insert Time: 0.1837 sec, Total Search Time: 0.045439 sec, Total Time: 0.2291 sec
Points: 210,000, Insert Time: 0.8210 sec, Total Search Time: 0.420408 sec, Total Time: 1.2414 sec
Points: 220,000, Insert Time: 0.1096 sec, Total Search Time: 0.129823 sec, Total Time: 0.2394 sec
Points: 320,000, Insert Time: 0.9145 sec, Total Search Time: 0.217231 sec, Total Time: 1.1317 sec
Points: 1,320,000, Build Time: 2.6356 sec, Total Search Time: 0.027646 sec, Total Time: 2.6632 sec
Points: 1,330,000, Insert Time: 0.0845 sec, Total Search Time: 0.196099 sec, Total Time: 0.2806 sec
Points: 1,340,000, Insert Time: 0.0837 sec, Total Search Time: 0.022029 sec, Total Time: 0.1057 sec
Points: 1,440,000, Insert Time: 0.8276 sec, Total Search Time: 0.454783 sec, Total Time: 1.2824 sec
Points: 1,540,000, Insert Time: 0.8421 

**------VP Dynamic Log Run 3----------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1416 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 6805.07it/s]


Total Search Time: 0.149121 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 119911.26it/s]


Insert Time: 0.0866 sec


...

Total Search Time: 0.834426 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 100170.86it/s]


Insert Time: 0.1030 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 38.03it/s]

Total Search Time: 2.962705 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1416 sec, Total Search Time: 0.149121 sec, Total Time: 0.2907 sec
Points: 110,000, Insert Time: 0.0866 sec, Total Search Time: 0.045514 sec, Total Time: 0.1321 sec
Points: 210,000, Insert Time: 0.8877 sec, Total Search Time: 0.387646 sec, Total Time: 1.2754 sec
Points: 220,000, Insert Time: 0.0858 sec, Total Search Time: 0.122001 sec, Total Time: 0.2078 sec
Points: 320,000, Insert Time: 0.8998 sec, Total Search Time: 0.198745 sec, Total Time: 1.0985 sec
Points: 1,320,000, Build Time: 2.5325 sec, Total Search Time: 0.086752 sec, Total Time: 2.6193 sec
Points: 1,330,000, Insert Time: 0.1016 sec, Total Search Time: 0.219993 sec, Total Time: 0.3216 sec
Points: 1,340,000, Insert Time: 0.0933 sec, Total Search Time: 0.023414 sec, Total Time: 0.1167 sec
Points: 1,440,000, Insert Time: 0.9607 sec, Total Search Time: 0.399740 sec, Total Time: 1.3605 sec
Points: 1,540,000, Insert Time: 0.9665 

**------VP Dyn Log Analysis-----**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 38.3814',
    'Total Time:: 38.8943',
    'Total Time:: 37.7327'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 38.3361 sec, Std Dev: 0.5821 sec, CV: 1.52%


-----------

**-----VP Dynamic Log Ratio Run 1-----**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1224 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 14251.80it/s]


Total Search Time: 0.004697 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 57759.73it/s]


Insert Time: 0.1762 sec


...

Total Search Time: 0.796502 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 119686.11it/s]


Insert Time: 0.0857 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 33.36it/s]

Total Search Time: 3.218568 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1224 sec, Total Search Time: 0.004697 sec, Total Time: 0.1271 sec
Points: 110,000, Insert Time: 0.1762 sec, Total Search Time: 0.043716 sec, Total Time: 0.2199 sec
Points: 210,000, Insert Time: 0.8175 sec, Total Search Time: 0.409911 sec, Total Time: 1.2274 sec
Points: 220,000, Insert Time: 0.0875 sec, Total Search Time: 0.122942 sec, Total Time: 0.2104 sec
Points: 320,000, Insert Time: 0.8296 sec, Total Search Time: 0.204884 sec, Total Time: 1.0345 sec
Points: 1,320,000, Build Time: 2.2651 sec, Total Search Time: 0.028139 sec, Total Time: 2.2932 sec
Points: 1,330,000, Insert Time: 0.0871 sec, Total Search Time: 0.198936 sec, Total Time: 0.2861 sec
Points: 1,340,000, Insert Time: 0.0855 sec, Total Search Time: 0.023196 sec, Total Time: 0.1087 sec
Points: 1,440,000, Insert Time: 0.8320 sec, Total Search Time: 0.460325 sec, Total Time: 1.2924 sec
Points: 1,540,000, Insert Time: 0.8159 

**--------VP Dyn Log Ratio Run 2-------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.1382 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 25070.56it/s]


Total Search Time: 0.002666 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 53188.67it/s]


Insert Time: 0.1911 sec


...

Total Search Time: 0.819549 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 120515.59it/s]


Insert Time: 0.0856 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 33.80it/s]

Total Search Time: 3.179865 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.1382 sec, Total Search Time: 0.002666 sec, Total Time: 0.1408 sec
Points: 110,000, Insert Time: 0.1911 sec, Total Search Time: 0.043789 sec, Total Time: 0.2348 sec
Points: 210,000, Insert Time: 0.8192 sec, Total Search Time: 0.410236 sec, Total Time: 1.2294 sec
Points: 220,000, Insert Time: 0.0835 sec, Total Search Time: 0.126742 sec, Total Time: 0.2103 sec
Points: 320,000, Insert Time: 0.8400 sec, Total Search Time: 0.212627 sec, Total Time: 1.0527 sec
Points: 1,320,000, Build Time: 2.4820 sec, Total Search Time: 0.026742 sec, Total Time: 2.5088 sec
Points: 1,330,000, Insert Time: 0.0846 sec, Total Search Time: 0.200974 sec, Total Time: 0.2855 sec
Points: 1,340,000, Insert Time: 0.0859 sec, Total Search Time: 0.022741 sec, Total Time: 0.1086 sec
Points: 1,440,000, Insert Time: 0.8496 sec, Total Search Time: 0.455340 sec, Total Time: 1.3049 sec
Points: 1,540,000, Insert Time: 0.8169 

**-------VP Dyn Log Ratio Run 3--------**

In [ ]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

epsilon_insert = 0.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.2683 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 9285.60it/s]


Total Search Time: 0.004820 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 115019.26it/s]


Insert Time: 0.0896 sec


...

Total Search Time: 0.839730 sec

Batch 20: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 97753.37it/s]


Insert Time: 0.1043 sec


Querying batch 20: 100%|██████████| 100/100 [00:02<00:00, 38.27it/s]

Total Search Time: 2.929699 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.2683 sec, Total Search Time: 0.004820 sec, Total Time: 0.2731 sec
Points: 110,000, Insert Time: 0.0896 sec, Total Search Time: 0.048622 sec, Total Time: 0.1382 sec
Points: 210,000, Insert Time: 0.8394 sec, Total Search Time: 0.395422 sec, Total Time: 1.2348 sec
Points: 220,000, Insert Time: 0.1080 sec, Total Search Time: 0.136467 sec, Total Time: 0.2445 sec
Points: 320,000, Insert Time: 0.8273 sec, Total Search Time: 0.207797 sec, Total Time: 1.0351 sec
Points: 1,320,000, Build Time: 2.4305 sec, Total Search Time: 0.080510 sec, Total Time: 2.5110 sec
Points: 1,330,000, Insert Time: 0.0986 sec, Total Search Time: 0.223662 sec, Total Time: 0.3222 sec
Points: 1,340,000, Insert Time: 0.0993 sec, Total Search Time: 0.032666 sec, Total Time: 0.1320 sec
Points: 1,440,000, Insert Time: 1.0148 sec, Total Search Time: 0.414944 sec, Total Time: 1.4297 sec
Points: 1,540,000, Insert Time: 0.9804 

**-----VP Dyn Log Ratio Analysis------**

In [ ]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 38.0691',
    'Total Time:: 38.6006',
    'Total Time:: 37.7821'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 38.1506 sec, Std Dev: 0.4153 sec, CV: 1.09%
